# API Request 2

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from xml.etree import ElementTree as et

In [2]:
gsid = pd.read_csv('school_gsid.csv')

In [3]:
Idlist = list(gsid['gsId'].unique().astype(str))
while '' in Idlist:
    Idlist = Idlist.remove('')

In [4]:
df_cols = ['Year', 'Sname', "Plan504", 'Percent Teachers InFirst SecondYear', 'ActSat Number Participation','Ethnicity',
           'Value', 'FreeAnd Reduced PriceLunch',"ApCourse Participation", "ApCourse Passed", "Id"]
out_df = pd.DataFrame()

for Id in Idlist[1:]:
    url = ("https://api.greatschools.org/school/census/CA/"+Id+"?key=***")
    r = requests.get(url)
    xml_data = r.content
    
    #parse xml data
    root = et.fromstring(r.text)
    tree = et.ElementTree(root)
    tree.write("census.xml")

    xtree = et.parse("census.xml")
    xroot = xtree.getroot()
    
    rows = []
       
    for name in xroot.findall('schoolName'):
        schoolName = name.text if name is not None else None
        
    for name in xroot.findall('plan504'):
        plan = name.text if name is not None else None
        
    for name in xroot.findall('percentTeachersInFirstSecondYear'):
        teachers = name.text if name is not None else None
        
    for name in xroot.findall('actSatNumberParticipation'):
        actSat = name.text if name is not None else None
        
    for name in xroot.findall('apCourseParticipation'):
        apCourseParticipation = name.text if name is not None else None
           
    for name in xroot.findall('apCoursePassed'):
        apCoursePassed = name.text if name is not None else None

    for name in xroot.findall('freeAndReducedPriceLunch'):
        freeAndReducedPriceLunch = name.text if name is not None else None
    
    for node in xroot.findall('.*/ethnicity'):
        year = node.find('year').text if node is not None else None
        ethnicity = node.find('name').text if node is not None else None
        value = node. find('value').text if node is not None else None
        rows.append({"Year": year, "Sname":schoolName, "Plan504": plan,
                     "Percent Teachers InFirst SecondYear": teachers,
                     "ActSat Number Participation": actSat, "Ethnicity": ethnicity, "Value": value, 
                     "FreeAnd Reduced PriceLunch": freeAndReducedPriceLunch,
                     "ApCourse Participation": apCourseParticipation, "ApCourse Passed": apCoursePassed,"Id":Id})

    temp_df = pd.DataFrame(rows, columns = df_cols)

    
    temp_df_pivot = pd.pivot_table(temp_df, index=['Year', 'Sname','Percent Teachers InFirst SecondYear','Plan504',
                                                   'ActSat Number Participation','FreeAnd Reduced PriceLunch',
                                                   'ApCourse Participation','ApCourse Passed', 'Id'],
                               columns='Ethnicity',values='Value', aggfunc=np.sum).reset_index()
    
    
    out_df = out_df.append(temp_df_pivot, ignore_index = True, sort=True)

In [6]:
out_df.head()

,ActSat Number Participation,ApCourse Participation,ApCourse Passed,Asian,"Black, non-Hispanic",FreeAnd Reduced PriceLunch,Hispanic,Id,Multiracial,Native American or Native Alaskan,Native Hawaiian or Other Pacific Islander,Percent Teachers InFirst SecondYear,Plan504,Sname,"White, non-Hispanic",Year,index
0,1071,46.8,885,45.63,6.52,18.16,11.87,1,3.96,0.17,0.76,7.9,1.5,Alameda High School,31.08,2016,NaN
1,25,0,885,70.29,4.0,30.29,8.57,14052,2.29,0.0,2.29,12.5,1.2,Alameda Science And Technology Institute,12.57,2016,NaN
2,25,0,885,0.0,0.0,30.29,0.0,17140,100.0,0.0,0.0,12.5,1.2,"Applied Scholastics Academy, East Bay",0.0,2016,NaN
3,178,0,885,35.63,17.51,40.34,16.69,6,6.66,0.3,1.42,1.8,2,Encinal Junior/Senior High,21.78,2016,NaN
4,4,0,885,10.26,18.8,46.15,39.32,9,7.69,1.71,2.56,10,3.9,Island High (Continuation) School,19.66,2016,NaN


In [7]:
out_df.shape

(2410, 17)

In [8]:
# exporting data
out_df.to_csv('census_data.csv')